# Memo: プロジェクト設計

ChatGPTと考えたもの

yamanori99  
October 7, 2024

# 設計を考える

以下のような設計を試みることが効果的です。

## プロジェクト構造の設計

プロジェクトは、以下のディレクトリ構造にすると良いでしょう。

``` default
project-root/
├── data/              # データセットやシミュレーションの結果
├── scripts/           # 各言語（C++, R, Python）のコード
├── analysis/          # データの分析用スクリプト
├── docs/              # レポート、論文、発表資料を出力する場所
├── references/        # 参考文献や先行研究
├── quarto/            # Quartoのテンプレートや設定ファイル
└── _quarto.yml        # Quartoのプロジェクト設定ファイル
```

-   data ディレクトリに実験やシミュレーションの結果を格納します。
-   scripts ディレクトリには各言語で書かれたシミュレーションコードを保存し、Quartoから動的に呼び出すことができるようにします。
-   analysis には分析用のコードを格納し、シミュレーション結果の分析や可視化を行います。
-   docs ディレクトリにはQuartoによって生成された文書（HTML, PDF, Slides）が保存されます。
-   references には文献や先行研究をまとめておき、簡単に参照できるようにします。
-   \_quarto.yml はプロジェクトの設定を管理するためのファイルです。

## 再現性のある文書

Quartoは動的なコードの実行結果を含む文書を作成することができるため、RやPythonのコードブロックをQuartoファイル内に直接埋め込みます。また、C++のコードは外部スクリプトとして保存し、Quartoからシェルコマンドで呼び出す形が良いでしょう。

## \_quarto.yml の設定

``` yml
project:
  title: "シミュレーション分析プロジェクト"
  author: "ユーザー名"
  output:
    html: 
      toc: true  # 目次
      code-fold: true  # コードの折りたたみ
    pdf: 
      toc: true
    docx: true
    beamer: true  # 発表資料

bibliography: references/references.bib  # 参考文献

execute:
  echo: true  # コード出力を表示
  warning: false  # 警告メッセージを非表示
  error: false  # エラーメッセージを非表示

format: 
  html: 
    theme: "flatly"  # テーマを指定
```

## 動的文書の作成

各言語のコードをQuartoファイル内に動的に埋め込むことで、実行時にシミュレーションや分析が行われます。例えば、Pythonのコードブロックは次のように書きます。

``` python
import numpy as np
# シミュレーションコード
results = np.random.rand(10)
print(results)
```

RやC++も同様に、Quartoを通して実行可能です。

## 複数の言語を統合

QuartoはRとPythonのコードを直接サポートしていますが、C++の場合は以下のように外部スクリプトを呼び出す方法が便利です。

``` zsh
g++ scripts/simulation.cpp -o simulation
./simulation
```

## バージョン管理

プロジェクト全体をGitで管理し、定期的に更新や改善を加えることを推奨します。また、GitHubなどで共同作業も行いやすくなります。

## 発表と論文執筆

QuartoはHTMLスライド、PDF、Wordドキュメントなど多様なフォーマットで出力できるため、プレゼン資料や論文のテンプレートを作成し、文書のスタイルやレイアウトを統一することができます。

このような設計でプロジェクトを進めると、シミュレーション結果の再現性を保ちながら、体系的かつ効率的に作業が進められます。

# パスの管理

Quartoでレンダリングを行う際に、ファイルのパスに問題が生じることはよくあります。これを防ぐためには、いくつかの工夫が必要です。ファイル構造を設計する際のポイントと、パスの問題を解決するための方法を以下にまとめます。

## 相対パスの使用

プロジェクト内のファイルやディレクトリにアクセスするときは、相対パスを使うことが基本です。Quartoプロジェクトはルートディレクトリからの相対パスを使うので、各ファイルやディレクトリはproject-rootを基準に指定します。

たとえば、次のようにスクリプトやデータファイルを参照します。

``` r
# Rコード内でデータを読み込む

data \<- read.csv("data/mydata.csv")
```

``` bash
# Bashで外部プログラムを実行する例

g++ scripts/simulation.cpp -o simulation ./simulation
```

## Quarto YAMLでの設定

Quartoのプロジェクト設定ファイル（\_quarto.yml）を活用し、ルートディレクトリやデフォルトパスを明確に定義します。これにより、各ドキュメント内でのパスの指定を簡素化できます。

project: type: website output-dir: docs/ \# 出力ファイルを保存する場所を明示

これにより、レンダリング時に出力されるHTMLやPDFファイルが自動的にdocs/ディレクトリに保存されます。

## hereパッケージの利用（Rの場合）

Rでは、hereパッケージを使ってプロジェクトのルートディレクトリを動的に検出し、そこから相対パスを指定する方法が便利です。

library(here)

data \<- read.csv(here(“data”, “mydata.csv”))

これにより、Quartoでの実行時にパスの問題を回避しやすくなります。hereを使うことで、ディレクトリ構造が複雑になっても、確実にファイルを参照できます。

## 外部プログラムのパスの設定

C++やPythonスクリプトを実行する際に、Quarto内で外部プログラムを呼び出す場合、絶対パスを明示することも一つの方法です。ただし、これにはファイルが動かないことを前提としています。プロジェクト全体を再配置する可能性がある場合、相対パスが安全です。

## パスのテスト

プロジェクトの規模が大きくなる前に、パスの問題を早期に発見して解決するために、各コードブロックでファイルのパスが正しく機能するかどうかを確認するテストを行うのも良い習慣です。

``` r
# Rでパスが通っているか確認するテスト

if (!file.exists("data/mydata.csv")) {
  stop("データファイルが見つかりません")
}
```

## Quartoのresourcesオプション

QuartoのYAMLでresourcesオプションを使うと、必要なファイルを特定のディレクトリに集めて処理することができます。これは特に複数のファイルを参照する場合に便利です。

``` yaml
format: 
  html: 
    resources: \["data/mydata.csv", "scripts/simulation.cpp"\]
```

これにより、ドキュメントのレンダリング時にこれらのリソースが確実に含まれるようになります。

これらの対策を講じることで、パスの問題

------------------------------------------------------------------------

# Cmakeを使用するときのC++コードの管理

CMakeを使用してC++のコードをコンパイルする場合、Quartoプロジェクト内での管理方法は以下のようにすることが考えられます。

## CMakeの設定ファイルを用意する

まず、CMakeを使用するために、CMakeLists.txtという設定ファイルをscriptsディレクトリ内に作成します。このファイルでは、どのソースファイルをコンパイルするか、どのライブラリをリンクするかなどを指定します。

scripts/CMakeLists.txtの例

``` cmake
cmake_minimum_required(VERSION 3.10)
project(Simulation)

# ソースファイルの指定
set(SOURCES simulation.cpp)

# 実行可能ファイルの作成
add_executable(simulation ${SOURCES})
```

## QuartoファイルからCMakeを呼び出す

Quartoプロジェクト内のRやPythonのコードブロックからCMakeを実行して、C++のプログラムをコンパイルします。以下は、その方法の例です。

Quartoファイル内でのCMakeの使用例

``` bash
# C++コードのコンパイル
# CMakeでプロジェクトをビルド
mkdir -p scripts/build
cd scripts/build
cmake ..
make
```

## 実行ファイルの呼び出し

コンパイルが完了したら、Quarto内で生成された実行可能ファイルを呼び出すことができます。以下のように、Bashのコードブロックを使って実行します。

``` bash
# 実行ファイルの呼び出し例
# コンパイルしたプログラムを実行
./scripts/build/simulation
```

## 全体のディレクトリ構造

最終的なディレクトリ構造は次のようになります。

``` default
project-root/
├── data/
├── scripts/
│   ├── CMakeLists.txt
│   ├── simulation.cpp
│   └── build/  # CMakeによって生成されるビルドディレクトリ
├── analysis/
├── docs/
├── references/
├── quarto/
└── _quarto.yml
```

## 注意点

-   CMakeを使用する場合、ビルドが初めて行われる際は、`scripts/build`ディレクトリを事前に作成する必要があります。これにより、ビルドアーティファクトが整理され、プロジェクトの構造が保たれます。

-   `CMakeLists.txt`を適切に設定することで、依存関係やコンパイルオプションを管理できます。

-   QuartoからCMakeを呼び出す際には、パスに注意して相対パスで指定することが重要です。

この方法を用いることで、Quartoプロジェクト内でCMakeによるC++コードの管理がスムーズに行えます。